In [1]:
import numpy as np

In [2]:
class Graph:
    
    def __init__(self, A: np.array):
        '''
        - A; the adjacency matrix
        '''
        
        assert (A.T == A).all()
        self.A = A
        self.N = A.shape[0]
        self.Q = np.diag(np.sum(self.A, axis=0)) - self.A
    
    def printAdjacency(self):
        '''
        print the adjacency matrix of the graph
        '''
        
        for i in range(self.N):
            print(str(A[i])[1:-1])
    
    def printLaplacian(self):
        '''
        print the laplacian matrix of the graph
        '''
        
        for i in range(self.N):
            print(str(self.Q[i])[1:-1])
            
    def complementaryAdjacency(self):
        '''
        return the complementarty adjacency matrix of the graph
        '''
        
        return np.ones(self.N) - np.identity(self.N) - self.A
    
    def khopwalk(self, k):
        '''
        - k: k-hop
        return the k-hop walks count of all nodes
        '''
        
        if k==0: 
            return 0
        A = self.A.copy()
        for _ in range(1,k):
            A = A.dot(A.T)
        return A
    
    def hopcount(self):
        '''
        return the distance matrix of the graph
        '''
        
        hopcount = self.A.copy()
        for k in range(2, self.N):
            khopwalk = self.khopwalk(k)
            hopcount[np.where(hopcount==0)] = (khopwalk[np.where(hopcount==0)] > 0) * k 
            if hopcount.all() != 0: 
                break
        return hopcount * (np.ones(hopcount.shape) - np.identity(self.N))
    
    def degree(self):
        '''
        return the degree matrix of all the nodes
        '''
        
        return np.sum(self.A, axis=0)
    
    def numberOfLink(self):
        '''
        return the total number of links in the graph
        '''
        
        return np.sum(self.degree()/2)
    
    def diameter(self):
        '''
        return the diameter of the graph which is the longest shortest path of the graph
        '''
        
        test = np.identity(self.N) + self.A
        poly = self.A.copy()
        rho = 1
        while(~test.all()):
            rho += 1
            poly = poly.dot(self.A)
            test += poly
        return rho
    
    def clusterCoef(self, v):
        '''
        - v: the vertex(node)
        return the clustering coefficient of vertex v
        '''
        
        degree_v = self.degree()[v]
        neigh = np.where(self.A[v]==1)[0]
        y = 0
        for n in neigh:
            neigh_neigh = np.where(self.A[n]==1)[0]
            for nn in neigh_neigh:
                if nn in neigh:
                    y += 1
        y = y/2
#         print("y =",y, "\t degree =",degree_v)
        return 2*y/(degree_v*(degree_v-1))

    def assortativity(self):
        '''
        return the degree assortativity of the graph
        '''
        
        N1 = np.sum(self.khopwalk(1))
        N2 = np.sum(self.khopwalk(2))
        N3 = np.sum(self.khopwalk(3))
        d = self.degree().copy().reshape(self.N,1)
        d3_sum = np.sum(d.T.dot(d).dot(d.T))
        return (N1*N3-N2**2)/(N1*d3_sum-N2**2)
    
    def spectraOfA(self):
        '''
        return all eigenvalues and eigenvectors of A
        '''
        lamb = -np.linalg.eig(self.A)[0]
        vec = np.linalg.eig(self.A)[1]
        
        vec = vec[:, np.argsort(lamb)]
        lamb = np.sort(lamb)
        
        return -lamb, vec
    
    def spectraOfQ(self):
        '''
        return all eigenvalues and eigenvectors of A
        '''
        mu = -np.linalg.eig(self.Q)[0]
        vec = np.linalg.eig(self.Q)[1]
        
        vec = vec[:, np.argsort(mu)]
        mu = np.sort(mu)
        
        return -mu, vec
    
    def triangleCount(self):
        '''
        return the total number of triangles in the graph
        '''
        
        return int(np.sum(self.spectraOfA()[0]**3)/6)
    
    def feature(self):
        '''
        print the following characteristics of a graph:
        * average degree
        * average hopcount
        * average clustering coefficient
        * average betweenness
        * degree assortativity
        '''
        
        avg_degree = np.sum(self.degree())/self.N
        avg_hopcount = np.mean(self.hopcount())
        avg_clustercoef = 0
        for v in range(self.N):
            avg_clustercoef += self.clusterCoef(v)
        avg_clustercoef = avg_clustercoef/self.N
        avg_betweenness = 1/self.numberOfLink() * self.N*(self.N-1)/2 * avg_hopcount
        
        print("average degree:", avg_degree)
        print("average hopcount:", avg_hopcount)
        print("average clustering coefficient:", avg_clustercoef)
        print("average betweenness:", avg_betweenness)
        print("degree assortativity:", self.assortativity())
        print("lambda 1:", self.spectraOfA()[0][0])
        print("algebraic connectivity:", self.spectraOfQ()[0][-2])
    
    def partitionFiedler(self, return_alpha=False):
        '''
        Graph partitioning into two disjoint subgraphs G1 and G2 
        according to Fiedler eigenvector
        - return_alpha: boolean indicating whehter return alpha 
        return:
        - gamma: the group index
        - R: the number of links between two groups
        '''
        
        zN_1 = self.spectraOfQ()[1][:,-2]
        gamma = np.sign(zN_1).reshape(self.N,1)
        R = gamma.T.dot(self.Q).dot(gamma) /4 
        alpha = gamma.T.dot(self.spectraOfQ()[1])
        
        if return_alpha:
            return gamma.reshape(-1,), int(R[0]), alpha
        return gamma.reshape(-1,), int(R[0])

In [3]:
A1 = np.array([[0,1,1,0,0,1], 
              [1,0,1,0,1,1], 
              [1,1,0,1,0,0], 
              [0,0,1,0,1,0], 
              [0,1,0,1,0,1], 
              [1,1,0,0,1,0]])

G1 = Graph(A1)
G1.feature()

average degree: 3.0
average hopcount: 1.1666666666666667
average clustering coefficient: 0.4166666666666666
average betweenness: 1.9444444444444444
degree assortativity: 0.44109808102345416
lambda 1: 3.114907541476758
algebraic connectivity: 1.6972243622680043


In [4]:
A2 = np.array([[0,1,0,1,1,1,1,1,1,0],
              [1,0,1,0,1,1,1,0,0,1],
              [0,1,0,0,1,1,1,0,0,0],
              [1,0,0,0,0,0,1,1,1,0],
              [1,1,1,0,0,1,0,0,0,0],
              [1,1,1,0,1,0,0,0,0,0],
              [1,1,1,1,0,0,0,0,0,0],
              [1,0,0,1,0,0,0,0,0,1],
              [1,0,0,1,0,0,0,0,0,0],
              [0,1,0,0,0,0,0,1,0,0]])
G2 = Graph(A2)
G2.feature()

average degree: 4.0
average hopcount: 1.46
average clustering coefficient: 0.5466666666666666
average betweenness: 3.285
degree assortativity: 0.43396518854910954
lambda 1: 4.489613051290812
algebraic connectivity: 1.384699252014049
